# Julia Workshop: Function Approximation

## @ CEF 2017

**Authors**: Chase Coleman and Spencer Lyon

**Date**: 27 June 2017


## Goal

There are two main interpolation packages that economists use in Julia: `BasisMatrices.jl` and `Interpolations.jl`.

* [BasisMatrices.jl](https://github.com/QuantEcon/BasisMatrices.jl)
  - Evaluates an approximated function at one (or many) points by building up the corresponding basis matrices
  - Many different types of interpolation (Chebyshev, complete, B-Splines, ...)
  - Portions of this code are based on the [CompEcon](http://www4.ncsu.edu/~pfackler/compecon/toolbox.html) package of Miranda and Fackler, so it should feel familiar for those who have previously used CompEcon.
* [Interpolations.jl](https://github.com/JuliaMath/Interpolations.jl)
  - Never directly stores a basis matrix
  - Only does B-splines (can mix and match degrees of approximation and allows one to choose boundary conditions)
  - Much of this code was written by people who work in image processing. There are lots of tricks that make it hard to beat performance wise.
  - Straightforward to evaluate an interpolator once you have it (overload of indexing methods).
  
This notebook should help you become familiar with what the two packages can do.

## BasisMatrices.jl

In [1]:
# Pkg.add("BasisMatrices")

In [2]:
using BasisMatrices

### Basic interpolation

`BasisMatrices` has a type called `Interpoland` which automates large portions of the behind the scenes details.

We show how to build and work with an `Interpoland` below:

#### Step 1: Build the initial grids

We call these the initial grids because certain interpolation types will add additional nodes (in particular, splines often need to add additional knots in order to ensure continuity of derivatives etc...).

In [3]:
xgrid0 = linspace(0.0, 2.0, 25)
ygrid0 = linspace(-1.0, 1.0, 10);

#### Step 2: Describe desired basis

In [4]:
xparams = SplineParams(xgrid0, 0, 2)  # Quadratic
yparams = SplineParams(ygrid0, 0, 3)  # Cubic

basis = Basis(xparams, yparams)

# Could also do:
# a_basis = Basis(aparams)
# y_basis = Basis(yparams)
# basis = Basis(a_basis, y_basis)

2 dimensional Basis on the hypercube formed by (0.0,-1.0) × (2.0,1.0).
Basis families are Spline × Spline


#### Step 3: Evaluate the function on the basis's nodes

We get the grid of points (including the points added by the package)

$$S = \begin{bmatrix} x_1 & y_1 \\ x_1 & y_2 \\ x_1 & \vdots \\ \vdots & \vdots \\ x_i & y_1 \\ x_i & \vdots \\ \vdots & \vdots \\ x_N & y_N \end{bmatrix}$$

Then evaluate the function $\sin(x) exp(y)$

In [5]:
S, (xgrid, ygrid) = nodes(basis)

f_vals = sin.(S[:, 1]) .* exp.(S[:, 2]);

#### Step 4: Create interpoland

In [6]:
itp1 = Interpoland(basis, f_vals);

#### Step 5: Evaluate interpoland

Single point

In [7]:
eval_1_pt = itp1([0.5, 0.75])
println(eval_1_pt - (sin(0.5)*exp(0.75)))

-2.153283957806451e-6


Many points

In [8]:
points = [0.0 -0.75
          0.5 -0.5
          0.75 0.25
          0.9 -.35]

eval_many_pt = itp1(points)
println(eval_many_pt - (sin.(points[:, 1]).*exp.(points[:, 2])))

[0.0; -1.19684e-6; -5.05038e-6; -2.28552e-6]


#### Step 6: Update coefficients (if needed)

In [9]:
f_vals2 = S[:, 1].^2 .* S[:, 2]

update_coefs!(itp1, f_vals2);

println(itp([1.5, 0.25]) - [sin(1.5)*exp(0.25), 1.5^2 * 0.25])

LoadError: UndefVarError: itp not defined

#### Note: Evaluate multiple functions at once

In [10]:
itp2 = Interpoland(basis, [f_vals f_vals2])

itp2(points)

4×2 Array{Float64,2}:
 0.0        0.0     
 0.290785  -0.125   
 0.875236   0.140625
 0.551999  -0.2835  

#### Note: Derivatives if necessary

In [11]:
itp1(points, [0 1])  # Take derivative with respect to y

4×1 Array{Float64,2}:
 0.0   
 0.25  
 0.5625
 0.81  

In [12]:
itp1(points, [0 2])  # 2nd derivative with respect to y

4×1 Array{Float64,2}:
  0.0        
 -1.91513e-15
 -1.33227e-15
 -1.33227e-15

In [13]:
itp1(points, [1 1])  # Cross partial wrt x and y

4×1 Array{Float64,2}:
 1.31731e-18
 1.0        
 1.5        
 1.8        

### Additional Information

### Directly building basis matrices

Sometimes, understanding how to build basis matrices on your own can speed up an algorithm.

We describe how one might do this. Begin by building up the initial grids and the basis type as in the previous examples.

In [14]:
xgrid0 = linspace(0.0, 5.0, 25)
ygrid0 = linspace(-2, 2, 10);

xparams = SplineParams(xgrid0, 0, 1)  # Linear
yparams = SplineParams(ygrid0, 0, 3)  # Cubic

basis = Basis(xparams, yparams)


2 dimensional Basis on the hypercube formed by (0.0,-2.0) × (5.0,2.0).
Basis families are Spline × Spline


#### Three "versions" of a basis matrix

These are described in more detail in the Miranda Fackler book, but we include short descriptions below.

1. `Expanded`: No clever tricks are used in building this up.
2. `Direct`: Computes the basis matrix
3. `Tensor`:

In [15]:
BasisMatrix(basis, Expanded(), S).vals[1]

312×300 sparse matrix with 2496 Float64 nonzero entries:
	[1  ,  51]  =  0.0703125
	[2  ,  51]  =  0.05625
	[3  ,  51]  =  0.028125
	[27 ,  51]  =  0.0330826
	[28 ,  51]  =  0.026466
	[29 ,  51]  =  0.013233
	[53 ,  51]  =  0.00260417
	[54 ,  51]  =  0.00208333
	[55 ,  51]  =  0.00104167
	[1  ,  52]  =  0.0
	⋮
	[311, 235]  =  0.0421875
	[312, 235]  =  0.028125
	[258, 236]  =  0.0
	[259, 236]  =  0.00104167
	[260, 236]  =  0.0015625
	[284, 236]  =  0.0
	[285, 236]  =  0.013233
	[286, 236]  =  0.0198495
	[310, 236]  =  0.0
	[311, 236]  =  0.028125
	[312, 236]  =  0.0421875

In [16]:
BasisMatrix(basis, Direct(), S).vals[2]

312×12 sparse matrix with 1248 Float64 nonzero entries:
	[1  ,   3]  =  0.0703125
	[2  ,   3]  =  0.0703125
	[3  ,   3]  =  0.0703125
	[4  ,   3]  =  0.0703125
	[5  ,   3]  =  0.0703125
	[6  ,   3]  =  0.0703125
	[7  ,   3]  =  0.0703125
	[8  ,   3]  =  0.0703125
	[9  ,   3]  =  0.0703125
	[10 ,   3]  =  0.0703125
	⋮
	[302,  10]  =  0.0703125
	[303,  10]  =  0.0703125
	[304,  10]  =  0.0703125
	[305,  10]  =  0.0703125
	[306,  10]  =  0.0703125
	[307,  10]  =  0.0703125
	[308,  10]  =  0.0703125
	[309,  10]  =  0.0703125
	[310,  10]  =  0.0703125
	[311,  10]  =  0.0703125
	[312,  10]  =  0.0703125

In [17]:
BasisMatrix(basis, Tensor()).vals[1]

25×25 sparse matrix with 50 Float64 nonzero entries:
	[1 ,  1]  =  1.0
	[1 ,  2]  =  0.0
	[2 ,  2]  =  1.0
	[3 ,  2]  =  2.66454e-16
	[2 ,  3]  =  0.0
	[3 ,  3]  =  1.0
	[4 ,  4]  =  1.0
	[4 ,  5]  =  0.0
	[5 ,  5]  =  1.0
	[6 ,  5]  =  1.06581e-15
	⋮
	[21, 20]  =  1.27898e-14
	[20, 21]  =  1.06581e-14
	[21, 21]  =  1.0
	[22, 22]  =  1.0
	[22, 23]  =  0.0
	[23, 23]  =  1.0
	[24, 23]  =  1.27898e-14
	[23, 24]  =  1.27898e-14
	[24, 24]  =  1.0
	[25, 24]  =  0.0
	[25, 25]  =  1.0

In [18]:
BasisMatrix(basis, Tensor()).vals[2]

12×12 sparse matrix with 48 Float64 nonzero entries:
	[1 ,  1]  =  1.0
	[2 ,  1]  =  0.296296
	[1 ,  2]  =  0.0
	[2 ,  2]  =  0.564815
	[3 ,  2]  =  0.25
	[1 ,  3]  =  0.0
	[2 ,  3]  =  0.132716
	[3 ,  3]  =  0.583333
	[4 ,  3]  =  0.166667
	[1 ,  4]  =  0.0
	⋮
	[11,  9]  =  0.00617284
	[12,  9]  =  0.0
	[9 , 10]  =  0.166667
	[10, 10]  =  0.583333
	[11, 10]  =  0.132716
	[12, 10]  =  0.0
	[10, 11]  =  0.25
	[11, 11]  =  0.564815
	[12, 11]  =  0.0
	[11, 12]  =  0.296296
	[12, 12]  =  1.0

### Complete polynomials

No nice type for interpolating with complete polynomials (yet), but can build basis matrices and evaluate.

In [19]:
_xgrid = repeat(xgrid0, inner=[length(ygrid0), 1])
_ygrid = repeat(ygrid0, outer=[length(xgrid0), 1])
grid = [_xgrid _ygrid]

ϕ = complete_polynomial(grid, 3)

250×10 Array{Float64,2}:
 1.0  0.0        0.0          0.0         …  -2.0       4.0        -8.0      
 1.0  0.0        0.0          0.0            -1.55556   2.41975    -3.76406  
 1.0  0.0        0.0          0.0            -1.11111   1.23457    -1.37174  
 1.0  0.0        0.0          0.0            -0.666667  0.444444   -0.296296 
 1.0  0.0        0.0          0.0            -0.222222  0.0493827  -0.0109739
 1.0  0.0        0.0          0.0         …   0.222222  0.0493827   0.0109739
 1.0  0.0        0.0          0.0             0.666667  0.444444    0.296296 
 1.0  0.0        0.0          0.0             1.11111   1.23457     1.37174  
 1.0  0.0        0.0          0.0             1.55556   2.41975     3.76406  
 1.0  0.0        0.0          0.0             2.0       4.0         8.0      
 1.0  0.208333   0.0434028    0.00904225  …  -2.0       4.0        -8.0      
 1.0  0.208333   0.0434028    0.00904225     -1.55556   2.41975    -3.76406  
 1.0  0.208333   0.0434028    0.0090422

In [20]:
f_vals3 = grid[:, 1].^2 .* grid[:, 2]
c = ϕ \ f_vals3

10-element Array{Float64,1}:
 -3.70913e-16
  6.43719e-16
 -2.50132e-16
  1.98498e-17
  1.0        
 -3.82269e-16
 -6.58252e-18
  5.01919e-16
 -3.95672e-17
 -3.99827e-17

In [21]:
dot(complete_polynomial([0.5, 0.5], 3), c)

0.12500000000000008

In [22]:
itp2([0.5, 0.5])[2]

0.12499999999999999

## Interpolations.jl

In [ ]:
# Pkg.add("Interpolations")

In [23]:
using Interpolations

### Interpolating with `Interpolations.jl`

`Interpolations.jl` is light in terms of memory usage because it never actually stores the basis matrices; instead, it computes each element on the fly.

#### Step 1: Build grids and evaluate function on grid

In [24]:
xgrid = linspace(0.0, 5.0, 25)
ygrid = linspace(-2.0, 2.0, 10);

f_vals = xgrid.^2 .* ygrid'  # Generates a 25x10 matrix

25×10 Array{Float64,2}:
  -0.0         -0.0         -0.0        …   0.0         0.0         0.0      
  -0.0868056   -0.0675154   -0.0482253      0.0482253   0.0675154   0.0868056
  -0.347222    -0.270062    -0.192901       0.192901    0.270062    0.347222 
  -0.78125     -0.607639    -0.434028       0.434028    0.607639    0.78125  
  -1.38889     -1.08025     -0.771605       0.771605    1.08025     1.38889  
  -2.17014     -1.68789     -1.20563    …   1.20563     1.68789     2.17014  
  -3.125       -2.43056     -1.73611        1.73611     2.43056     3.125    
  -4.25347     -3.30826     -2.36304        2.36304     3.30826     4.25347  
  -5.55556     -4.32099     -3.08642        3.08642     4.32099     5.55556  
  -7.03125     -5.46875     -3.90625        3.90625     5.46875     7.03125  
  -8.68056     -6.75154     -4.82253    …   4.82253     6.75154     8.68056  
 -10.5035      -8.16937     -5.83526        5.83526     8.16937    10.5035   
 -12.5         -9.72222     -6.94444    

#### Step 2: Create interpolator object

Notice that we don't give it the grids that it is on. By default `Interpolations.jl` pretends all grids are equally spaced with a distance of 1 between them -- Here it would think that the x dimension goes between 1 and 25 and the y dimension goes between 1 and 10.

In [25]:
itp4 = interpolate(f_vals, BSpline(Cubic(Natural())), OnGrid());

# Can have different degrees of approximation if wanted using below code
# itp = interpolate(f_vals, (BSpline(Linear()), BSpline(Cubic(Natural()))),
#                   OnGrid());

Bottom right element of `f_vals`

In [26]:
itp4[25, 10]

50.0

#### Step 3: Scale the interpolator object

We want to move the grid back to the scales we have. In order to extract extra speed, it is assumed everything is equally spaced. To help enforce this, they require that whatever it is scaled by is a `linspace` of some sort.

In [27]:
itp4_s = scale(itp4, collect(xgrid), collect(ygrid))  # Won't work

LoadError: MethodError: no method matching scale(::Interpolations.BSplineInterpolation{Float64,2,Array{Float64,2},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line}},Interpolations.OnGrid,1}, ::Array{Float64,1}, ::Array{Float64,1})[0m
Closest candidates are:
  scale(::AbstractArray{T,2}, ::AbstractArray{T,1}) at deprecated.jl:49
  scale([1m[31m::Number[0m, ::AbstractArray{T,N}) at deprecated.jl:49
  scale(::AbstractArray{T,N}, [1m[31m::Number[0m) at deprecated.jl:49
  ...[0m

In [28]:
itp4_s = scale(itp4, xgrid, ygrid);  # Will work

In [29]:
itp4_s[5.0, 2.0]  # Bottom right element again

50.0

#### Step 4: Evaluate

`Interpolations.jl` only allows for evaluation at one point at a time.

In [30]:
f_interp_values = Array{Float64}(100, 100)
x_fine_grid = linspace(xgrid[1], xgrid[end], 100)
y_fine_grid = linspace(ygrid[1], ygrid[end], 100)

for (ix, x) in enumerate(x_fine_grid)
    for (iy, y) in enumerate(y_fine_grid)
        f_interp_values[ix, iy] = itp4_s[x, y]
    end
end

f_true_values = x_fine_grid.^2 .* y_fine_grid'

println(maximum(abs, f_interp_values - f_true_values))

0.008074808933957733


#### Note: Gradient if needed

In [31]:
gradient(itp4_s, 1.0, 2.0)

2-element Array{Float64,1}:
 3.99955
 1.00001